In [1]:
import pandas as pd

In [2]:
arq = "estimativa_dou_2023.csv"
url = "https://raw.githubusercontent.com/hjort/populacao-municipal/main/" + arq

!test -f $arq || wget -O $arq $url
!ls -lah $arq

-rw-r--r-- 1 s033236209 domain users 195K jun  3 00:10 estimativa_dou_2023.csv


In [3]:
!head $arq

2023;RO;11;00015;Alta Floresta D'Oeste;21169
2023;RO;11;00379;Alto Alegre dos Parecis;12898
2023;RO;11;00403;Alto Paraíso;22826
2023;RO;11;00346;Alvorada D'Oeste;12465
2023;RO;11;00023;Ariquemes;112685
2023;RO;11;00452;Buritis;41878
2023;RO;11;00031;Cabixi;4539
2023;RO;11;00601;Cacaulândia;6228
2023;RO;11;00049;Cacoal;85343
2023;RO;11;00700;Campo Novo de Rondônia;14326


In [7]:
dfm = pd.read_csv(
    arq, sep=';',
    names=['ano', 'uf', 'cod1', 'cod2', 'nome', 'habitantes'],
    dtype={'cod1': 'object', 'cod2': 'object'}
)
dfm

,ano,uf,cod1,cod2,nome,habitantes
0,2023,RO,11,00015,Alta Floresta D'Oeste,21169
1,2023,RO,11,00379,Alto Alegre dos Parecis,12898
2,2023,RO,11,00403,Alto Paraíso,22826
3,2023,RO,11,00346,Alvorada D'Oeste,12465
4,2023,RO,11,00023,Ariquemes,112685
...,...,...,...,...,...,...
5565,2023,GO,52,22302,Vila Propício,6061
5566,2023,GO,52,00175,Água Fria de Goiás,5959
5567,2023,GO,52,00209,Água Limpa,1737
5568,2023,GO,52,00258,Águas Lindas de Goiás,235479


In [8]:
dfm['codigo'] = dfm[['cod1', 'cod2']].apply(lambda x: int(x[0] + x[1]), axis=1)
dfm.drop(['ano', 'cod1', 'cod2', 'habitantes'], axis=1, inplace=True)
dfm.rename({'nome': 'municipio'}, axis=1, inplace=True)
dfm = dfm[['codigo', 'uf', 'municipio']]
dfm.set_index('codigo', inplace=True)
dfm

,uf,municipio
codigo,,
1100015,RO,Alta Floresta D'Oeste
1100379,RO,Alto Alegre dos Parecis
1100403,RO,Alto Paraíso
1100346,RO,Alvorada D'Oeste
1100023,RO,Ariquemes
...,...,...
5222302,GO,Vila Propício
5200175,GO,Água Fria de Goiás
5200209,GO,Água Limpa


In [9]:
import re

def convert_special_chars(text):
    special_chars = {
        'ÃÁÂÀÄÅ': 'A',
        'ÈÉÊË': 'E',
        'ÌÍÎÏ': 'I',
        'ÒÓÔÕÖ': 'O',
        'ÙÚÜÛ': 'U',
        'ÝŸ': 'Y',
        'Ç': 'C',
        'Ñ': 'N',
        '\'': ''
    }
    for special_char, ascii_equivalent in special_chars.items():
        text = re.sub('[' + special_char + ']', ascii_equivalent, text)
    return text

# Example usage:
text = "This is a string with special characters: àáâãäå æ ç èéêë ìíîï ñ òóôõöø œ ùúûü ýÿ".upper()
converted_text = convert_special_chars(text)
print(converted_text)

THIS IS A STRING WITH SPECIAL CHARACTERS: AAAAAA Æ C EEEE IIII N OOOOOØ Œ UUUU YY


In [10]:
dfm['municipio'].apply(lambda x: x.upper())

codigo
1100015      ALTA FLORESTA D'OESTE
1100379    ALTO ALEGRE DOS PARECIS
1100403               ALTO PARAÍSO
1100346           ALVORADA D'OESTE
1100023                  ARIQUEMES
                    ...           
5222302              VILA PROPÍCIO
5200175         ÁGUA FRIA DE GOIÁS
5200209                 ÁGUA LIMPA
5200258      ÁGUAS LINDAS DE GOIÁS
5300108                   BRASÍLIA
Name: municipio, Length: 5570, dtype: object

In [11]:
dfm['municipio'].apply(lambda x: convert_special_chars(x.upper()))

codigo
1100015       ALTA FLORESTA DOESTE
1100379    ALTO ALEGRE DOS PARECIS
1100403               ALTO PARAISO
1100346            ALVORADA DOESTE
1100023                  ARIQUEMES
                    ...           
5222302              VILA PROPICIO
5200175         AGUA FRIA DE GOIAS
5200209                 AGUA LIMPA
5200258      AGUAS LINDAS DE GOIAS
5300108                   BRASILIA
Name: municipio, Length: 5570, dtype: object

In [12]:
dfm['municipio2'] = dfm['municipio'].apply(lambda x: convert_special_chars(x.upper()))
dfm

,uf,municipio,municipio2
codigo,,,
1100015,RO,Alta Floresta D'Oeste,ALTA FLORESTA DOESTE
1100379,RO,Alto Alegre dos Parecis,ALTO ALEGRE DOS PARECIS
1100403,RO,Alto Paraíso,ALTO PARAISO
1100346,RO,Alvorada D'Oeste,ALVORADA DOESTE
1100023,RO,Ariquemes,ARIQUEMES
...,...,...,...
5222302,GO,Vila Propício,VILA PROPICIO
5200175,GO,Água Fria de Goiás,AGUA FRIA DE GOIAS
5200209,GO,Água Limpa,AGUA LIMPA


In [13]:
csv = 'municipios.csv'
dfm.to_csv(csv)

In [14]:
!head $csv && tail $csv

codigo,uf,municipio,municipio2
1100015,RO,Alta Floresta D'Oeste,ALTA FLORESTA DOESTE
1100379,RO,Alto Alegre dos Parecis,ALTO ALEGRE DOS PARECIS
1100403,RO,Alto Paraíso,ALTO PARAISO
1100346,RO,Alvorada D'Oeste,ALVORADA DOESTE
1100023,RO,Ariquemes,ARIQUEMES
1100452,RO,Buritis,BURITIS
1100031,RO,Cabixi,CABIXI
1100601,RO,Cacaulândia,CACAULANDIA
1100049,RO,Cacoal,CACOAL
5221858,GO,Valparaíso de Goiás,VALPARAISO DE GOIAS
5221908,GO,Varjão,VARJAO
5222005,GO,Vianópolis,VIANOPOLIS
5222054,GO,Vicentinópolis,VICENTINOPOLIS
5222203,GO,Vila Boa,VILA BOA
5222302,GO,Vila Propício,VILA PROPICIO
5200175,GO,Água Fria de Goiás,AGUA FRIA DE GOIAS
5200209,GO,Água Limpa,AGUA LIMPA
5200258,GO,Águas Lindas de Goiás,AGUAS LINDAS DE GOIAS
5300108,DF,Brasília,BRASILIA
